In [2]:
import pandas as pd

path = "./data/all_data.csv"
df = pd.read_csv(path)

In [7]:
df.head()

,Unnamed: 0,Perimetre,Nature,Date,Heures,y,Thermique,Nucleaire,Eolien,Solaire,Hydraulique,Pompage,Bioenergies,Ech. physiques
0,0,Ile-de-France,Donn�es d�finitives,2016-01-01,00:00,8885.0,293.0,-,18.0,0.0,10.0,-,137.0,8428.0
1,1,Ile-de-France,Donn�es d�finitives,2016-01-01,00:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Ile-de-France,Donn�es d�finitives,2016-01-01,00:30,8709.0,309.0,-,16.0,0.0,10.0,-,148.0,8226.0
3,3,Ile-de-France,Donn�es d�finitives,2016-01-01,00:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Ile-de-France,Donn�es d�finitives,2016-01-01,01:00,8343.0,309.0,-,13.0,0.0,10.0,-,149.0,7864.0


In [10]:
df_cons = df[["Heures","y"]]
for r in df_cons:
    if df_cons['y']:
        

,Heures,y
0,00:00,8885.0
1,00:15,NaN
2,00:30,8709.0
3,00:45,NaN
4,01:00,8343.0
...,...,...
175387,22:45,NaN
175388,23:00,10180.0
175389,23:15,NaN
175390,23:30,10019.0
